In [68]:
import re
import pandas as pd
import requests
import urllib
import json
from string import ascii_letters, digits
import time
import urllib

In [69]:
def parse_card_list():
    f = open("card_list.txt", "r")
    data = f.readlines()
    f.close()
    dataset = [[data[i].strip('//').strip(),
                data[i + 1].strip('//').strip().split()[0].split(':')[0],
                data[i + 1].strip('//').strip().split()[0].split(':')[0].strip('[').strip(']'),
                data[i + 2].strip('')] for i in range(0, len(data), 4)]
    df = pd.DataFrame(dataset, columns=['Name', 'Index', 'ID', 'File'])
    df = df.replace(r'\n',' ', regex=True) 
    df_er = df[(df['File'].str.lower().str.contains("token")) & (~df['Name'].str.lower().str.contains("token"))]
    return df, df_er

In [70]:
df_org, df_er = parse_card_list()

In [71]:
#df_org.to_csv('card_list.csv', index=False)
#df_er.to_csv('card_list_er.csv', index=False, encoding='utf-8')

In [ ]:
"url = "https://db.ygoprodeck.com/api/v7/cardinfo.php?name="
rs = []
err_name = []
for name in df_er['Name']:
    response = requests.get(url + urllib.parse.quote(name.encode('utf-8')))
    if response.status_code == 200:
        rs.append(response.json())
    else:
        err_name.append(name)
    time.sleep(1)

with open("rs.json", "w") as outfile:
    outfile.write(json.dumps(rs, indent=4))
with open("rs_err.json", "w") as outfile:
    outfile.write(json.dumps(err_name, indent=4))

In [72]:
f = open('rs.json')
data = json.load(f)
f.close()

In [73]:
dt = {item['data'][0]['name']: {
    'card_code': [it['set_code'] for it in item['data'][0]['card_sets']] if 'card_sets' in item['data'][0] else [],
    'image_url': item['data'][0]['card_images'][0]['image_url'],
    'image_url_small': item['data'][0]['card_images'][0]['image_url_small'],
    'desc': item['data'][0]['desc'],
} for item in data}

In [89]:
count = 0
count_null = 0
for name, card in dt.items():
    pre_proc = []
    f = False
    for code in card['card_code']:
        w = 0
        p1, p2 = code.split('-')
        p1a = p1
        if len(p1a) > 3:
            p1a = re.sub('[0-9]', '', p1)
            w += 0.5
        p2a = p2.lstrip('EN').lstrip('E')
        if len(p1a) == 3 and len(p2a) <= 3 and p2a.isnumeric():
            f = True
            if any(map(str.isdigit, p1a)):
                w += 0.5
            pre_proc.append((p1a, p2a, w))
        else:
            p1b = p1a
            if len(p1a) != 3:
                p1b = p1.replace('0', '')
                w += 1
            p2b = p2a
            if not (len(p2a) <= 3 and p2a.isnumeric()):
                p2b = p2.lstrip(ascii_letters)
                w += 1
            if len(p1b) == 3 and len(p2b) <= 3 and p2b.isnumeric():
                f = True
                pre_proc.append((p1b, p2b, w))
            else:
                p1c = p1b
                if len(p1c) > 3:
                    if not p1c[-1].isnumeric():
                        p1c = p1b[0:-1]
                        w += 1
                if len(p1c) == 3 and len(p2b) <= 3 and p2b.isnumeric():
                    f = True
                    pre_proc.append((p1c, p2b, w))
                else:
                    w = 100
                    pre_proc.append((p1, p2, w))
    if len(card['card_code']) == 0:
        card['file'] = ''
        count_null += 1
    else:
        tmp = None
        for item in pre_proc:
            if (tmp is None) or (item[2] < tmp[2]):
                tmp = item
        card['file'] = tmp[0] + tmp[1].zfill(3) + '.bmp'
    if not f and len(card['card_code']) != 0:
        count += 1

In [90]:
count_null

151

In [ ]:
for n in df[df['File'] == '']['Name']:
    response = requests.get(url + urllib.parse.quote(name.encode('utf-8')))
    if response.status_code == 200:
        rs.append(response.json())

In [111]:
response = requests.get("https://yugipedia.com/api.php?action=parse&page=" + "Aqua Snake".replace(' ', '_'))

In [113]:
response.text

'<html>\r\n<head><title>403 Forbidden</title><script type="text/javascript">!(function(o,_name){o[_name]=o[_name]||function $(){($.q=$.q||[]).push(arguments)},o[_name].v=o[_name].v||2;!(function(o,t,n,f,e,i){function c(f,e){(f=(function(t,n){try{if(n=(t=o.localStorage).getItem("_aQS02NTY3QTBBMkYxNkI4MjZERENFNEJDNDctNA"))return JSON.parse(n).lgk||[];if((t.getItem("v4ac1eiZr0")||"").split(",")[4]>0)return[[_name+"-engaged","true"]]}catch(f){}})())&&typeof f.forEach===n&&(e=o[t].pubads())&&f.forEach((function(o){o&&o[0]&&e.setTargeting(o[0],o[1]||"")}))}(i=o[t]=o[t]||{}).cmd=i.cmd||[],typeof i.pubads===n?c():typeof i.cmd.unshift===n?i.cmd.unshift(c):i.cmd.push(c)})(window,"googletag","function");;})(window,decodeURI(\'a%64m%69%72%61l\'));!(function(t,c,i){i=t.createElement(c),t=t.getElementsByTagName(c)[0],i.async=1,i.src="https://possiblepencil.com/chunks/387a751e9ddb36cc00f96dc9bab4af81c9c7-prod.js",t.parentNode.insertBefore(i,t)})(document,"script");</script></head>\r\n<body bgcolor="w

In [ ]:
dt["Aqua Snake"]["file"] = "B05003"
dt["Acid Crawler"]["file"] = "VL5001"
dt["Fiend Reflection #1"]["file"] = "VL4020"
dt["Arlownay"]["file"] = "VL7004"
dt["Arma Knight"]["file"] = "VL7005"
dt["Fairy of the Fountain"]["file"] = "DL4095"
dt["Living Vase"]["file"] = "VL3017"
dt["Minomushi Warrior"]["file"] = "BR2026"
dt["Horn Imp"]["file"] = "VL6022"
dt["Wing Eagle"]["file"] = ""
dt["Wing Egg Elf"]["file"] = ""
dt["Weather Control"]["file"] = ""
dt["Rock Ogre Grotto #2"]["file"] = ""
dt["Water Element"]["file"] = ""
dt["Water Girl"]["file"] = ""
dt["Beautiful Beast Trainer"]["file"] = ""
dt["Wood Clown"]["file"] = ""
dt["Waterdragon Fairy"]["file"] = ""
dt["Candle of Fate"]["file"] = ""
dt["Air Eater"]["file"] = ""
dt["Eldeen"]["file"] = ""
dt["Fairywitch"]["file"] = ""
dt["Ghoul with an Appetite"]["file"] = ""
dt["Tao the Chanter"]["file"] = ""
dt["Akihiron"]["file"] = ""
dt["Spirit of the Winds"]["file"] = ""
dt["Djinn the Watcher of the Wind"]["file"] = ""
dt["Kanikabuto"]["file"] = ""
dt["Ganigumo"]["file"] = ""
dt["Kamakiriman"]["file"] = ""
dt["Masked Clown"]["file"] = ""
dt["Crow Goblin"]["file"] = ""
dt["Garvas"]["file"] = ""
dt["Boulder Tortoise"]["file"] = ""
dt["Rock Spirit"]["file"] = ""
dt["Barrel Rock"]["file"] = ""
dt["Cyber Soldier"]["file"] = ""
dt["Kattapillar"]["file"] = ""
dt["Monstrous Bird"]["file"] = ""
dt["Beaked Snake"]["file"] = ""
dt["Togex"]["file"] = ""
dt["Grappler"]["file"] = ""
dt["Krokodilus"]["file"] = ""
dt["Twin Long Rods #1"]["file"] = ""
dt["Gorgon Egg"]["file"] = ""
dt["Phantom Ghost"]["file"] = ""
dt["Ice Water"]["file"] = ""
dt["Ancient Sorcerer"]["file"] = ""
dt["Eyearmor"]["file"] = ""
dt["Cyber Commander"]["file"] = ""
dt["Korogashi"]["file"] = ""
dt["Torike"]["file"] = ""
dt["Mech Bass"]["file"] = ""
dt["Phantom Dewan"]["file"] = ""
dt["Ancient Tree of Enlightenment"]["file"] = ""
dt["Lisark"]["file"] = ""
dt["The Wandering Doomed"]["file"] = ""
dt["Zarigun"]["file"] = ""
dt["30,000-Year White Turtle"]["file"] = ""
dt["Sea King Dragon"]["file"] = ""
dt["Tatsunootoshigo"]["file"] = ""
dt["Fiend's Hand"]["file"] = ""
dt["Dokuroizo the Grim Reaper"]["file"] = ""
dt["Genin"]["file"] = ""
dt["Abyss Flower"]["file"] = ""
dt["Emperor of the Land and Sea"]["file"] = ""
dt["Stone Ghost"]["file"] = ""
dt["Mystical Capture Chain"]["file"] = ""
dt["Ill Witch"]["file"] = ""
dt["Lord of Zemia"]["file"] = ""
dt["Solitude"]["file"] = ""
dt["Mech Mole Zombie"]["file"] = ""
dt["Dark Chimera"]["file"] = ""
dt["Dark Shade"]["file"] = ""
dt["Dark Plant"]["file"] = ""
dt["Turtle Raccoon"]["file"] = ""
dt["Ancient Jar"]["file"] = ""
dt["Dice Armadillo"]["file"] = ""
dt["Orion the Battle King"]["file"] = ""
dt["One Who Hunts Souls"]["file"] = ""
dt["Winged Egg of New Life"]["file"] = ""
dt["Lunar Queen Elzaim"]["file"] = ""
dt["Giant Scorpion of the Tundra"]["file"] = ""
dt["The Drdek"]["file"] = ""
dt["Serpent Marauder"]["file"] = ""
dt["Spiked Snail"]["file"] = ""
dt["Graveyard and the Hand of Invitation"]["file"] = ""
dt["Job-Change Mirror"]["file"] = ""
dt["Tentacle Plant"]["file"] = ""
dt["Tenderness"]["file"] = ""
dt["Toad Master"]["file"] = ""
dt["Dragon Statue"]["file"] = ""
dt["Doriado"]["file"] = ""
dt["Dorover"]["file"] = ""
dt["Doron"]["file"] = ""
dt["The Melting Red Shadow"]["file"] = ""
dt["The Thing That Hides in the Mud"]["file"] = ""
dt["Nightmare Scorpion"]["file"] = ""
dt["Night Lizard"]["file"] = ""
dt["Mystery Hand"]["file"] = ""
dt["Fiend Sword"]["file"] = ""
dt["Bio Plant"]["file"] = ""
dt["Bat"]["file"] = ""
dt["Hurricail"]["file"] = ""
dt["Man-Eating Plant"]["file"] = ""
dt["Hitodenchak"]["file"] = ""
dt["Hyo"]["file"] = ""
dt["Fire Eye"]["file"] = ""
dt["Synchar"]["file"] = ""
dt["Vishwar Randi"]["file"] = ""
dt["Binding Chain"]["file"] = ""
dt["Boo Koo"]["file"] = ""
dt["Kappa Avenger"]["file"] = ""
dt["Gate Deeg"]["file"] = ""
dt["Lucky Trinket"]["file"] = ""
dt["Pot the Trick"]["file"] = ""
dt["Bone Mouse"]["file"] = ""
dt["Bolt Escargot"]["file"] = ""
dt["Holograh"]["file"] = ""
dt["White Dolphin"]["file"] = ""
dt["Mountain Warrior"]["file"] = ""
dt["Fungi of the Musk"]["file"] = ""
dt["Mabarrel"]["file"] = ""
dt["Barrel Lily"]["file"] = ""
dt["Machine Attacker"]["file"] = ""
dt["Madjinn Gunn"]["file"] = ""
dt["Wicked Dragon with the Ersatz Head"]["file"] = ""
dt["Embryonic Beast"]["file"] = ""
dt["Midnight Fiend"]["file"] = ""
dt["B. Dragon Jungle King"]["file"] = ""
dt["Muse-A"]["file"] = ""
dt["The Bewitching Phantom Thief"]["file"] = ""
dt["Mechaleon"]["file"] = ""
dt["Megirus Light"]["file"] = ""
dt["Supporter in the Shadows"]["file"] = ""
dt["Wood Remains"]["file"] = ""
dt["Monsturtle"]["file"] = ""
dt["Mon Larvas"]["file"] = ""
dt["Yashinoki"]["file"] = ""
dt["Yamatano Dragon Scroll"]["file"] = ""
dt["Spirit of the Mountain"]["file"] = ""
dt["Yormungarde"]["file"] = ""
dt["Armored Rat"]["file"] = ""
dt["Zanki"]["file"] = ""
dt["LaMoon"]["file"] = ""
dt["LaLa Li-oon"]["file"] = ""
dt["Rainbow Marine Mermaid"]["file"] = ""
dt["Leo Wizard"]["file"] = ""
dt["Akakieisu"]["file"] = ""
dt["Royal Guard"]["file"] = ""
dt["Laughing Flower"]["file"] = ""

In [80]:
df = pd.DataFrame([[name, dt[name]['file'], dt[name]['image_url'],
                   dt[name]['image_url_small'], dt[name]['desc']] for name in dt],
                 columns=['Name', 'File', 'Img', 'Imgs', 'Desc'])

In [81]:
df_rs = pd.merge(df_er, df, how='left', on=['Name'], suffixes=("1", "2"))
df_rs['File'] = df_rs.pop('File2').fillna(df_rs.pop('File1'))

In [93]:
#for n in df[df['File'] == '']['Name']:
#    print('dt["%s"]["file"] = ""' % n)

In [94]:
df[['Name', 'File', 'Desc']]

,Name,File,Desc
0,Rhaimundos of the Red Sword,AP8014.bmp,"Armed with the Sword of Crimson Flames, this w..."
1,Aqua Snake,,This monster hypnotizes an opponent with the b...
2,Aqua Dragon,MDP013.bmp,"""Fairy Dragon"" + ""Amazon of the Seas"" + ""Zone ..."
3,Unknown Warrior of Fiend,SDK017.bmp,The speed of this warrior creates an intense v...
4,Beastly Mirror Ritual,PRC002.bmp,"This card is used to Ritual Summon ""Fiend's Mi..."
...,...,...,...
314,Key Mace #2,WCP005.bmp,This fiend brings out the worst in everyone.
315,Laughing Flower,,A flower whose laugh can confuse the minds of ...
316,Obelisk the Tormentor,EGO001.bmp,Requires 3 Tributes to Normal Summon (cannot b...
317,Slifer the Sky Dragon,EGS001.bmp,Requires 3 Tributes to Normal Summon (cannot b...


In [95]:
df[df['File'] == ''].shape

(151, 5)

In [99]:
test = df[df['File'] != '']

In [100]:
test[test.duplicated()]

,Name,File,Img,Imgs,Desc
